## 1. Tải Dữ Liệu từ CSV

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModel
import numpy as np
import torch
from datasets import load_dataset
import torch.nn as nn
import os
from typing import List
from tqdm import tqdm


os.environ["CUDA_VISIBLE_DEVICES"] = "1" ## Setup CUDA GPU 1



In [3]:

class BERTIntentClassification(nn.Module):


    def __init__(self, model_name="bert-base-uncased", num_classes=10, dropout_rate=0.1, cache_dir = "huggingface"):
        super(BERTIntentClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name, cache_dir = cache_dir)
        # Get BERT hidden size
        hidden_size = self.bert.config.hidden_size
        self.ffnn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_classes)
        )


    def freeze_bert(self):
        for param in self.bert.parameters():
            param.requires_grad = False


    def get_pooling(self, hidden_state, attention_mask):
        """
        Get mean pooled representation from BERT hidden states
        Args:
            hidden_state: BERT output containing hidden states
        Returns:
            pooled_output: Mean pooled representation of the sequence
        """
        # Get last hidden state
        last_hidden_state = hidden_state.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]

        if attention_mask is not None:
            # Expand attention mask to match hidden state dimensions
            attention_mask = attention_mask.unsqueeze(-1)  # [batch_size, seq_len, 1]

            # Mask out padding tokens
            masked_hidden = last_hidden_state * attention_mask

            # Calculate mean (sum / number of actual tokens)
            sum_hidden = torch.sum(masked_hidden, dim=1)  # [batch_size, hidden_size]
            count_tokens = torch.sum(attention_mask, dim=1)  # [batch_size, 1]
            pooled_output = sum_hidden / count_tokens
        else:
            # If no attention mask, simply take mean of all tokens
            pooled_output = torch.mean(last_hidden_state, dim=1)

        return pooled_output


    def forward(self, input_ids, attention_mask, **kwargs):
        """
        Forward pass of the model
        Args:
            input_ids: Input token IDs
            attention_mask: Attention mask for padding
        Returns:
            logits: Raw logits for each class
        """
        # Get BERT hidden states
        hidden_state = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        # Get pooled representation
        hidden_state_pooling = self.get_pooling(hidden_state=hidden_state, attention_mask=attention_mask)

        # Pass through FFNN classifier
        logits = self.ffnn(hidden_state_pooling)

        return logits


In [4]:
class TrainerCustom(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs

        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss


# 1. Load Dataset and with Dataloader

In [ ]:






# # Bước 1: Tải dữ liệu
# # Sử dụng dataset sẵn có từ Hugging Face hoặc tải từ file cục bộ
# dataset = load_dataset("imdb", cache_dir = "huggingface")  # Ví dụ: Dữ liệu IMDB để phân loại sentiment
# # Thay thế trường 'text' thành 'input_ids' trong train_dataset và test_dataset
# def preprocess_dataset(dataset):
#     return dataset.map(lambda example: {
#             "input_ids": example['text'],
#             "label": example['label']
#         },
#         remove_columns=["text"],
#         num_proc=4  # Sử dụng 4 tiến trình song song để xử lý nhanh hơn
#     )

# train_dataset = preprocess_dataset(dataset["train"])
# test_dataset = preprocess_dataset(dataset["test"])



In [6]:
# print(train_dataset)
# # Truy cập mẫu cụ thể
# train_sample = train_dataset[:10]
# test_sample = test_dataset[:2]
# print(train_sample)


# from datasets import Dataset

# train_sample = train_dataset[:10]

# # Chuyển từ dict về Dataset
# train_sample_dataset = Dataset.from_dict(train_sample)
# test_sample_dataset = Dataset.from_dict(test_sample)
# print(train_sample_dataset)
# print(type(train_sample_dataset))
# # Output: <class 'datasets.arrow_dataset.Dataset'>


# # In thử 1 hàng trong test_sample_dataset
# print("First row in test_sample_dataset:")
# print(test_sample_dataset[0])




In [7]:
from datasets import Dataset

def load_csv_dataset(csv_path, text_column, label_column):
    """
    Tải dataset từ file CSV và đổi tên cột.

    Args:
        csv_path (str): Đường dẫn đến file .csv.
        text_column (str): Tên cột chứa văn bản.
        label_column (str): Tên cột chứa nhãn.

    Returns:
        Dataset: Tập dữ liệu đã tải từ file .csv.
    """
    # Tải dữ liệu từ file .csv
    dataset = Dataset.from_csv(csv_path)
    # Đổi tên cột
    dataset = dataset.rename_columns({text_column: "input_ids", label_column: "label"})
    return dataset

# Sử dụng hàm
csv_path = "/content/chatbot_intent_data_v1_En.csv"             # Đường dẫn file CSV
text_column = "input_ids"       # Cột chứa văn bản
label_column = "label"        # Cột chứa nhãn

# Tải dataset
dataset = load_csv_dataset(csv_path, text_column, label_column)

# Kiểm tra dữ liệu
print(dataset)

# Truy cập mẫu cụ thể
sample_dataset = dataset.select(range(10))  # Lấy 10 mẫu đầu tiên
print(sample_dataset)


# In thử 1 hàng trong test_sample_dataset
print("First row in test_sample_dataset:")
print(sample_dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['label', 'input_ids'],
    num_rows: 27
})
Dataset({
    features: ['label', 'input_ids'],
    num_rows: 10
})
First row in test_sample_dataset:
{'label': 'Agree', 'input_ids': 'Yes, I want to show you the picture.'}


In [8]:
def check_invalid_samples(dataset):
    invalid_samples = []
    for idx, sample in enumerate(dataset):
        if not isinstance(sample["input_ids"], str) or sample["input_ids"].strip() == "":
            invalid_samples.append((idx, sample))
    return invalid_samples

# Kiểm tra dữ liệu không hợp lệ
invalid_samples = check_invalid_samples(dataset)
print("\n===== Invalid Samples =====")
print(invalid_samples)



===== Invalid Samples =====
[]


In [9]:
# Tự động phát hiện nhãn và tạo ánh xạ nhãn
def create_label_mapping(dataset_list):
    """
    Tự động phát hiện tất cả các nhãn từ danh sách dataset và ánh xạ chúng thành số nguyên.
    """
    all_labels = set()
    for dataset in dataset_list:
        all_labels.update(dataset["label"])  # Tập hợp tất cả các nhãn từ dataset

    label_to_int = {label: idx for idx, label in enumerate(sorted(all_labels))}
    print(f"Ánh xạ nhãn: {label_to_int}")
    return label_to_int

# Hàm chuyển đổi nhãn
def preprocess_labels(example, label_to_int):
    example["label"] = label_to_int.get(example["label"], -1)  # Gán -1 cho nhãn không hợp lệ
    return example

# Tạo ánh xạ nhãn
label_mapping = create_label_mapping([dataset])

# Áp dụng chuyển đổi nhãn
dataset = dataset.map(lambda example: preprocess_labels(example, label_mapping))

# Kiểm tra kết quả
print(dataset)

# Truy cập mẫu cụ thể
sample_dataset = dataset.select(range(10))  # Lấy 10 mẫu đầu tiên
print(sample_dataset)

# In thử 1 hàng trong sample_dataset
print("First row in sample_dataset:")
print(sample_dataset[0])

Ánh xạ nhãn: {'Agree': 0, 'Decline': 1, 'Fallback': 2, 'Silence': 3, 'Uncertain': 4}


Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids'],
    num_rows: 27
})
Dataset({
    features: ['label', 'input_ids'],
    num_rows: 10
})
First row in sample_dataset:
{'label': 0, 'input_ids': 'Yes, I want to show you the picture.'}


In [16]:
def split_dataset(dataset, test_size=0.2, seed=42):
    """
    Chia dataset thành tập train và test.

    Args:
        dataset (Dataset): Tập dữ liệu đầy đủ.
        test_size (float): Tỷ lệ dữ liệu test (0.0 - 1.0).
        seed (int): Seed để chia dữ liệu ngẫu nhiên.

    Returns:
        tuple: (train_dataset, test_dataset) - Tập train và test.
    """
    if not (0.0 < test_size < 1.0):
        raise ValueError("test_size phải nằm trong khoảng (0.0, 1.0)")
    if len(dataset) < 2:
        raise ValueError("Dataset phải có ít nhất 2 mẫu để chia.")

    train_test_split = dataset.train_test_split(test_size=test_size, seed=seed)
    print(f"Chia dataset: {len(train_test_split['train'])} mẫu train, {len(train_test_split['test'])} mẫu test")
    return train_test_split["train"], train_test_split["test"]

# Chia dataset
train_dataset, test_dataset = split_dataset(dataset, test_size=0.2)

# Kiểm tra dữ liệu
print("Train dataset:", train_dataset)
print("Test dataset:", test_dataset)

# Truy cập mẫu cụ thể
sample_train_dataset = train_dataset.select(range(5))  # Lấy 10 mẫu đầu tiên từ train
sample_test_dataset = test_dataset.select(range(2))    # Lấy 10 mẫu đầu tiên từ test

print("Sample train dataset:", sample_train_dataset)
print("Sample test dataset:", sample_test_dataset)

Chia dataset: 21 mẫu train, 6 mẫu test
Train dataset: Dataset({
    features: ['label', 'input_ids'],
    num_rows: 21
})
Test dataset: Dataset({
    features: ['label', 'input_ids'],
    num_rows: 6
})
Sample train dataset: Dataset({
    features: ['label', 'input_ids'],
    num_rows: 5
})
Sample test dataset: Dataset({
    features: ['label', 'input_ids'],
    num_rows: 2
})


# 2. Tokenizer

In [10]:





# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir = "huggingface")
model = BERTIntentClassification(
    model_name=model_name,
    num_classes=5
)
model.freeze_bert() # Froze Layer BERT
max_seq_length = 512


def collate_fn(features):
    inputs = []
    labels = []
    for element in features:
        inputs.append(element.get("input_ids"))
        labels.append(element.get("label"))

    labels = torch.tensor(labels, dtype=torch.long)

    token_inputs = tokenizer(
        inputs,
        add_special_tokens=True,
        truncation=True,
        padding=True,
        max_length=max_seq_length,
        return_overflowing_tokens=False,
        return_length=False,
        return_tensors="pt",
    )
    token_inputs.update({
        "labels": labels,
    })
    return token_inputs


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

# 3. Train Model

## 3.1 Log Wandb

In [11]:
!pip install --upgrade wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 40.6 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.1
    Uninstalling wandb-0.19.1:
      Successfully uninstalled wandb-0.19.1


In [12]:
!pip install python-dotenv

In [13]:
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy key từ biến môi trường
wandb_api_key = os.getenv("WANDB_API_KEY")
print(wandb_api_key[:5])

c8767


In [14]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=os.getenv("WANDB_API_KEY"))


wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Cách thiết lập thông qua TrainingArguments
Khi sử dụng Trainer, bạn có thể đặt tên dự án trực tiếp trong TrainingArguments bằng cách sử dụng tham số report_to và run_name. Tuy nhiên, để đặt project, bạn cần khởi tạo một phiên wandb trước hoặc truyền cấu hình này thông qua wandb.init().

Điều chỉnh TrainingArguments:
```python
training_args = TrainingArguments(
    output_dir="./results_",          # Thư mục lưu kết quả
    eval_strategy="epoch",           # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",            # Thư mục lưu log
    logging_strategy="steps",        # Log theo steps
    logging_steps=10,                # Log sau mỗi 10 bước
    save_strategy="epoch",           # Lưu checkpoint sau mỗi epoch
    save_total_limit=3,              # Lưu tối đa 3 checkpoint
    report_to="wandb",               # Báo cáo log tới wandb
    run_name="bert_run_1"            # Tên phiên chạy trên wandb
)
```

## 3.2 Train

https://docs.wandb.ai/guides/integrations/huggingface/

```python
from transformers import TrainingArguments, Trainer
```
với việc LOG vào WANDB


Sau khi huấn luyện xong model, bạn có thể lưu các models vào artifact

```pthon
# # Lưu mô hình vào wandb
# import os
# model_dir = "./results_"  # Thư mục chứa mô hình
# wandb.init(
#     project="bert-intent-classification",  # Tên dự án
#     name="bert_run_1"                     # Tên phiên chạy
# )
# # Tạo một artifact và lưu mô hình vào wandb
# artifact = wandb.Artifact(
#     name="bert-intent-classification-model",  # Tên artifact
#     type="model",                             # Loại artifact (ví dụ: "model")
#     description="Fine-tuned BERT for intent classification"
# )

# # Thêm các file mô hình vào artifact
# artifact.add_dir("./results_")  # Thư mục chứa checkpoint
# wandb.log_artifact(artifact)

# # Kết thúc phiên wandb
# wandb.finish()
```

Tuy nhiên, ko hài lòng với cách này. Mình ORDER GPT SEARCH WEB VÀ CHỈ CẦN CHẠY DÒNG WANDB_LOG_MODEL này 1 cái là ngon luôn.

```
import os
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

```

In [82]:

# # Bước 6: Cài đặt tham số huấn luyện
# training_args = TrainingArguments(
#     output_dir="./results_",          # Thư mục lưu kết quả
#     eval_strategy="epoch",    # Đánh giá sau mỗi epoch
#     learning_rate=2e-4,
#     per_device_train_batch_size=128,
#     per_device_eval_batch_size=128,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=None,
#     logging_strategy = "epoch",
#     save_strategy="epoch",          # Lưu trọng số sau mỗi epoch
#     save_total_limit=3,
# )

# # Bước 7: Tạo Trainer
# trainer = TrainerCustom(
#     model=model,
#     args=training_args,
#     train_dataset=sample_train_dataset,
#     eval_dataset=sample_test_dataset,
#     tokenizer=tokenizer,
#     data_collator = collate_fn,
# )

# # Bước 8: Huấn luyện
# trainer.train()






<ipython-input-82-c398dfa5e582>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TrainerCustom.__init__`. Use `processing_class` instead.
  trainer = TrainerCustom(


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not SequenceClassifierOutput

In [ ]:

import wandb

# Khởi tạo wandb
wandb.init(
    project="bert-intent-classification",  # Tên dự án
    name="bert_run_1"                     # Tên phiên chạy
)


# Bước 6: Cài đặt tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",          # Thư mục lưu kết quả
    eval_strategy="epoch",    # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=None,
    logging_strategy = "epoch",
    save_strategy="epoch",          # Lưu trọng số sau mỗi epoch
    save_total_limit=3,
    report_to="wandb",
    run_name="bert_run_1"
)

# Bước 7: Tạo Trainer
trainer = TrainerCustom(
    model=model,
    args=training_args,
    train_dataset=sample_train_dataset,
    eval_dataset=sample_test_dataset,
    tokenizer=tokenizer,
    data_collator = collate_fn,
)

# Bước 8: Huấn luyện
trainer.train()

# Kết thúc phiên wandb
wandb.finish()




<ipython-input-26-54aeb818dadd>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TrainerCustom.__init__`. Use `processing_class` instead.
  trainer = TrainerCustom(


Epoch,Training Loss,Validation Loss
1,0.206300,No log
2,0.282800,No log
3,0.238600,No log
4,0.215900,No log
5,0.168800,No log


wandb: Adding directory to artifact (./results/checkpoint-1)... Done. 2.4s
wandb: Adding directory to artifact (./results/checkpoint-2)... Done. 2.8s
wandb: Adding directory to artifact (./results/checkpoint-3)... Done. 12.4s
wandb: Adding directory to artifact (./results/checkpoint-4)... Done. 2.5s
wandb: Adding directory to artifact (./results/checkpoint-5)... Done. 53.7s
wandb: Adding directory to artifact (./results/checkpoint-5)... Done. 3.0s


In [64]:
# Bước 9: Đánh giá trên tập kiểm tra
trainer.evaluate()

{'eval_runtime': 0.1695,
 'eval_samples_per_second': 11.802,
 'eval_steps_per_second': 5.901,
 'epoch': 5.0}

# Inference

In [73]:
sentence = "What is the weather like today?"


inputs = tokenizer(
    sentence,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=512
)



In [74]:
model.eval()  # Đặt mô hình ở chế độ đánh giá (không tính gradient)
with torch.no_grad():  # Không cần tính gradient
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()  # Lấy nhãn dự đoán
    print(f"Predicted class: {predicted_class}")


Predicted class: 0
